In [1]:
import pandas as pd
import argparse
import logging
import json
import os
import ast
import pathlib
import fiona
import numpy as np
import geopandas as gpd
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
from geopandas.tools import geocode
from shapely.geometry import Point
import matplotlib.pyplot as plt
import requests
from zipfile import ZipFile
from tqdm import trange, tqdm
from operator import itemgetter

In [2]:
# Get the typical size of a MFH in each district - works

def get_hh_MFH(DATA_DIRECTORY,CANTON_CSV_DIRECTORY,UST_district, REA_CODES_DESIRED_LC):

    TI_hh = pd.DataFrame()
    district_hh = pd.DataFrame()
    ewid_count = pd.DataFrame()

    for dis_num in UST_district:
        communes = UST_comune.loc[UST_comune["Numero del Distretto"] == dis_num, "Numero UST del Comune"]

        for com_num in communes:
            #print("comune: ", com_num)

            fileloc =  CANTON_CSV_DIRECTORY

            filename_gdf = "raw-gdf-"+str(com_num)+".csv"
            filename_data = "DATA_"+str(com_num)+".csv"

            if pathlib.Path(fileloc/filename_gdf).exists():
                data_res = pd.read_csv(fileloc/filename_gdf , header=0, index_col=0)
                #data_res = data_res_dirty.drop_duplicates(keep="first")
            elif pathlib.Path(fileloc/filename_data).exists():
                data_res = pd.read_csv(fileloc/filename_data , header=0, index_col=0)
                #data_res = data_res_dirty.drop_duplicates(keep="first")

            data_res = data_res.loc[data_res["gstat"] == 1004]
            data_res = data_res.loc[(data_res["gkat"] == 1020) | (data_res["gkat"] == 1030)]
            data_res = data_res[REA_CODES_DESIRED_LC]

            data_res["gklas_fill"] =  data_res["gklas"]
            data_res = data_res.reset_index(drop=True)

            for g in range(len(data_res)):
                 if (np.isnan(data_res.at[g,"gklas"])):
                    if (data_res.at[g,"gkat"] == 1020):
                        data_res.loc[g,"gklas_fill"] = 1110
                    elif (data_res.at[g,"gkat"] == 1030):
                        data_res.loc[g,"gklas_fill"] = 1122

            data_res["egid_duplicate"] = data_res.duplicated(subset = ["egid"], keep = False)

            no_nan = data_res.dropna(subset = ["ewid"])
            no_nan = no_nan.reset_index(drop=True) #,, inplace=True

            for g in range(len(no_nan)):
                ewid_count.at[g,"ewid"] = ast.literal_eval(no_nan.at[g,"ewid"]) 

            no_nan["ewid_len"] = ewid_count.ewid.apply(lambda x: len(x))

            duplicates = no_nan.loc[no_nan["egid_duplicate"] == True]
            group = duplicates.groupby("egid")["ewid_len"].sum()
            group = pd.DataFrame(group)

            # Replace ewid_len values in DataFrame A based on DataFrame B
            no_nan['ewid_len'] = no_nan.apply(lambda row: group.loc[row['egid'], 'ewid_len'] if row['egid'] in group.index else row['ewid_len'], axis=1)
            no_nan.drop_duplicates(subset =["egid"], keep="first", inplace = True)

            district_hh = pd.concat([district_hh,no_nan])

        abi_num = district_hh [["egid","gklas_fill","ewid_len"]]
        abi_num = abi_num.groupby(["gklas_fill","ewid_len"]).count()
        abi_num = abi_num.rename(columns={"egid":"count"})
        abi_num.rename_axis(index={"gklas_fill":"gklas","ewid_len":"number of households"}, inplace = True)
        med = pd.DataFrame(abi_num).reset_index(level="number of households")
        med["TOT"] = med["number of households"]*med["count"]
        MFH = med.loc[1122.0]
        MFH_avghh = MFH["TOT"].sum()/MFH["count"].sum()

        DISTRICT = str(UST_comune.loc[(UST_comune["Numero del Distretto"] == dis_num), "Nome del Distretto"].iloc[0])
        hh = pd.DataFrame([MFH_avghh], index=["MFH_avg_households"], columns=[DISTRICT])

        TI_hh = pd.concat([TI_hh,hh], axis=1)
        district_hh = district_hh[0:0]

    TI_hh["TI"] = TI_hh.mean(axis=1)
    return TI_hh

In [3]:
def generate_number_list(num):
    number_list = [str(i) for i in range(1, num + 1)]
    return number_list

In [4]:
# Get the ration between primaty and secondary households - works
def get_hh_stats(DATA_DIRECTORY,CANTON_CSV_DIRECTORY,UST_district):

    sup_file = DATA_DIRECTORY / "support_data" 

    data_households_2020 = pd.read_excel(sup_file/"household_inventory_2020-10_2056.xlsx", sheet_name="ZWG_2020_Q3" , header=0, index_col=0)
    data_households_2023 = pd.read_excel(sup_file/"household_inventory_2023-03_2056.xlsx", sheet_name="ZWG_2023" , header=0, index_col=0)
    
    ## GET ratios for primary households and difference between 2023 and 2020 - WORKS!

    TI_hh_2020 = pd.DataFrame()
    district_hh_2020 = pd.DataFrame()
    district_hh_2023 = pd.DataFrame()

    for dis_num in UST_district:
        communes = UST_comune.loc[UST_comune["Numero del Distretto"] == dis_num, "Nome del Comune"]    
        DISTRICT = str(UST_comune.loc[(UST_comune["Numero del Distretto"] == dis_num), "Nome del Distretto"].iloc[0])

        for com_name in communes:
            hh_2020 = data_households_2020.loc[data_households_2020["Name"] == com_name]#
            hh_2020 = hh_2020.reset_index()

            hh_2023 = data_households_2023.loc[data_households_2023["Name"] == com_name]#
            hh_2023 = hh_2023.reset_index()

            district_hh_2020 = pd.concat([district_hh_2020, hh_2020])
            district_hh_2023 = pd.concat([district_hh_2023, hh_2023])

        dis_tot_hh_2020 = district_hh_2020["ZWG_3150"].sum()
        dis_tot_primary_2020 = district_hh_2020["ZWG_3010"].sum() + district_hh_2020["ZWG_3100"].sum()

        calc_dist_secondary_2020 = round(((dis_tot_hh_2020 - dis_tot_primary_2020)/dis_tot_hh_2020),3)

        ratio_secondary_mean_2020 = district_hh_2020["ZWG_3120"].mean()
        ratio_secondary_median_2020 = district_hh_2020["ZWG_3120"].median()

        dis_tot_hh_2023 = district_hh_2023["ZWG_3150"].sum()
        dis_tot_primary_2023 = district_hh_2023["ZWG_3010"].sum() + district_hh_2023["ZWG_3100"].sum()

        hh_diff = round((dis_tot_hh_2020/dis_tot_hh_2023),3)

        d_hh = pd.DataFrame([calc_dist_secondary_2020, hh_diff], index=["hh2020_p_ratio", "hh2023_diff"], columns=[DISTRICT])

        TI_hh_2020 = pd.concat([TI_hh_2020,d_hh], axis=1)
        district_hh_2020 = district_hh_2020[0:0]
        district_hh_2023 = district_hh_2023[0:0]

    return TI_hh_2020

In [5]:
def get_pop(communes):
    com_pop = data_pop[data_pop["Num_Comune"].isin(communes)]# GGDENR Numéro OFS de la commune district_limits[district_limits['NAME'].isin(TI_districts)]
    com_pop = com_pop.reset_index(drop =True)
    d_pop = com_pop["Stato della popolazione al 31 dicembre"].sum()
    com_pop["percent of disrict pop"] = round((com_pop["Stato della popolazione al 31 dicembre"]/d_pop),2)
    
    return com_pop

In [6]:
def get_coordinates(comune):
    base_url = "https://api3.geo.admin.ch/rest/services/api/SearchServer"
    params = {
        "searchText": comune,
        "type": "locations",
        "lang": "en",
        "limit": 1,
    }

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Check for any request errors
        data = response.json()

        # Extract longitude and latitude from the first result
        if "results" in data and len(data["results"]) > 0:
            result = data["results"][0]
            longitude = result["attrs"]["lon"]
            latitude = result["attrs"]["lat"]
            return latitude, longitude
        else:
            print("No results found for Lugano.")
            return None, None

    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None, None

In [7]:
## 1 - COMMON INFO

## DEFINE CONSTANTS 
REA_CODES_DESIRED_LC = ['egid', 'strname_deinr', 'ggdename', 'ggdenr',
       'gexpdat', 'gdekt', 'egrid','gebnr', 'gkode', 'gkodn', 'gksce', 'gstat', 'gkat', 'gklas',
       'gbauj', 'gbaup', 'gabbj', 'garea', 'gastw', 'gazzi', 'gebf', 'gwaerzh1',
       'genh1', 'gwaersceh1', 'gwaerdath1', 'gwaerzh2', 'genh2', 'gwaersceh2',
       'gwaerdath2', 'gwaerzw1', 'genw1', 'gwaerscew1', 'gwaerdatw1',
       'gwaerzw2', 'genw2', 'gwaerscew2', 'gwaerdatw2',"ewid"]

EPSG_CODE = "EPSG:2056"
GKLAS = [1110,1121,1122,1130, 1211, 1212, 1220, 1230, 1251, 1261, 1262, 1263, 1264, 1265, 1272, 1241, 1242, 1271, 1274, 1275, 1276, 1277, 1278, 1231, 1252, 1273]
GKLAS_HTG = [1110, 1121, 1122, 1130, 1211, 1212, 1220, 1230, 1251, 1261, 1262, 1263, 1264, 1265, 1272, 1231]
GKLAS_NOHTG = [1241, 1242, 1252, 1271, 1273, 1274, 1275, 1276, 1277, 1278]
GKLAS_SFH = 1110
GKLAS_DFH = 1121
GKLAS_MFH = 1122
GKLAS_RES = [GKLAS_SFH, GKLAS_DFH, GKLAS_MFH]
GBAUP = [8011,8012,8013,8014,8015,8016,8017,8018,8019,8020,8021,8022,8023]
GSTAT = 1004 # existing

DICT = {"SFH":1110.0, "DFH":1121.0, "MFH":1122.0, "HABITAT_COMMUNAUTAIRE":1130.0, "HOTEL":1211.0, "HEBERGEMENT":1212.0, "OFFICE":1220.0, "COMMERCIAL":1230.0, "RESTO_BAR":1231.0, "TRANSP_STATIONS":1241.0, "GARAGE": 1242.0, 
        "INDUSTRIAL":1251.0, "RESERVOIRS":1252.0, "CULTURAL":1261.0, "MUSEUM_LIBRARY":1262.0, "ACADEMIC":1263.0, "HOSPITAL":1264.0, "SPORTS":1265.0, "AGRICULTURE":1271.0, "RELIGIOUS":1272.0, "ANCIENT":1273.0, "OTHER_PUBLIC":1274.0, "OUTSIDE": 1275.0,
        "ANIMALS":1276.0, "GREENHOUSE":1277.0, "AGRI_STORAGE":1278.0}

RES_code = itemgetter("SFH", "DFH", "MFH")(DICT)
AGR_code = itemgetter("AGRICULTURE")(DICT) #, "ANIMALS", "GREENHOUSE", "AGRI_STORAGE"
IND_code = itemgetter("INDUSTRIAL")(DICT)
SER_code = itemgetter("HABITAT_COMMUNAUTAIRE","HOTEL", "HEBERGEMENT", "OFFICE", "COMMERCIAL", "RESTO_BAR", "TRANSP_STATIONS","CULTURAL", "MUSEUM_LIBRARY", "ACADEMIC", "HOSPITAL", "SPORTS","RELIGIOUS")(DICT)
OTHER_code = itemgetter("GARAGE","RESERVOIRS", "ANCIENT", "OUTSIDE", "OTHER_PUBLIC")(DICT)

CANTON = "Ticino"

### Consumi di energia (in gigawattora), secondo la destinazione e il vettore energetico, in Ticino, nel 2020
S1_AGR_elec = 61640 # kWh/y = consumo "Altro (cantieri, agricoltura ecc.)" 2020 / Aziende primario 2020
S2_IND_elec = 153650 #kWh/y consumo "Artigianato e industria"/ Aziende secondario 2020
S3_SER_elec = 22180 #kWh/y consumo "Commercio e servizi" /Aziende terzario 2020
AVG_HH_ELEC_TI = 3499 # kWh/y USTAT 2020 = tot apparecchi/tot abitazioni ewid 


AVG_DHW_PP = 40 # L/DAY/PERSON ODIS
AVG_M2_PP = 49.1 #TI - Superficie media per occupante, per Cantone, 2021, https://www.bfs.admin.ch/bfs/it/home/statistiche/costruzioni-abitazioni/abitazioni/condizioni-abitazione/superficie-persona.html
SUPERFICIE_MEDIA = 97.8 #https://www.bfs.admin.ch/bfs/it/home/statistiche/costruzioni-abitazioni/abitazioni/dimensioni.html
MFH_PP = 2.1 #https://www.bfs.admin.ch/bfs/it/home/statistiche/costruzioni-abitazioni/abitazioni/condizioni-abitazione/densita-utilizzazione.html
SFH_PP = 2.7
NET_ERA_SHARE = 0.9 # SIA 416 - la superficie netta di un locale o un gruppo di locali corrisponde ca. al 90% della superficie del piano [SIA 380-4 (2006)]
UNOCC = 0.0283 # should be around 0.0283 for TI

## SET LOCATION OF FILES
pathlib.Path()
NOTEBOOK_PATH = pathlib.Path().resolve()
p = NOTEBOOK_PATH.parent
DATA_DIRECTORY = p / "data"
MAP_DIRECTORY = DATA_DIRECTORY /'maps'
BLDG_DIRECTORY = DATA_DIRECTORY /'Buildings_rea'
CANTON = "Ticino"
CANTON_CSV_DIRECTORY = BLDG_DIRECTORY / CANTON
sup_file = DATA_DIRECTORY / "support_data" 

## READ SUPPORT DATA
data_sh = pd.read_csv(sup_file/"demand_SH.csv" , header=0, index_col=0)
data_dhw = pd.read_csv(sup_file/"demand_DHW.csv", header=0, index_col=0)
fehh = pd.read_csv(sup_file/"fehh.csv", header=0, index_col=0)
HDD =  pd.read_csv(sup_file/"district_pop_weighted_HDD.csv", sep=";", header=0, index_col=0)
data_s123 = pd.read_csv(sup_file/"S1_S2_S3.csv" , header=0, index_col=0)
# Read the shp file and decode the Geopandas dataframe using the Swiss coordinates (epsg code: 2056)
SWISSTOPO_DISTRICT_FILE = "swissboundaries3d_2023-01_2056_5728.shp/swissBOUNDARIES3D_1_4_TLM_BEZIRKSGEBIET.shp" #shapefile downloaded from https://www.swisstopo.admin.ch/fr/geodata/landscape/boundaries3d.html
SWISSTOPO_CANTON_FILE = "swissboundaries3d_2023-01_2056_5728.shp/swissBOUNDARIES3D_1_4_TLM_KANTONSGEBIET.shp" #shapefile downloaded from https://www.swisstopo.admin.ch/fr/geodata/landscape/boundaries3d.html. NOTE: The actual shapefile (.shp) is useless without the companion files: .dbf, .shx, .prj etc..

UST_file = "UST_comune.csv"
canton ="TI"
UST_comune = pd.read_csv(MAP_DIRECTORY/UST_file)
UST_comune = UST_comune[UST_comune['Cantone'] == canton].reset_index()
UST_district = UST_comune['Numero del Distretto'].drop_duplicates().reset_index(drop=True)
avg_hh_MFH = get_hh_MFH(DATA_DIRECTORY,CANTON_CSV_DIRECTORY,UST_district, REA_CODES_DESIRED_LC)
hh_stats = get_hh_stats(DATA_DIRECTORY,CANTON_CSV_DIRECTORY,UST_district)

### AGR, IND, SER

In [25]:
# 4 - Calculate the heating and electricty demand for agriculture, industry, and services - WORKS!

## CREATE EMPTY DATAFRAMES
TI_nonres = pd.DataFrame()
district_nonres = pd.DataFrame()
S123 = pd.DataFrame()

for dis_num in UST_district:  
    DISTRICT = str(UST_comune.loc[(UST_comune["Numero del Distretto"] == dis_num), "Nome del Distretto"].iloc[0])
    print(DISTRICT)
    
    dis_S1_tot_elec = data_s123.loc[DISTRICT, "S1_Aziende"]*S1_AGR_elec
    dis_S2_tot_elec = data_s123.loc[DISTRICT, "S2_Aziende"]*S2_IND_elec
    dis_S3_tot_elec = data_s123.loc[DISTRICT, "S3_Aziende"]*S3_SER_elec
    tot_elec = pd.DataFrame(np.array([[dis_S1_tot_elec], [dis_S2_tot_elec], [dis_S3_tot_elec]]),  columns = [DISTRICT], index = [ "agr_elec", "ind_elec", "ser_elec"])
    S123 = pd.concat([S123,tot_elec], axis = 1)
 
    ic = len(UST_comune.loc[(UST_comune["Numero del Distretto"] == dis_num)])


    for com_i in range(ic):
        com_name = UST_comune.loc[UST_comune["Numero del Distretto"] == dis_num, "Nome del Comune"].iloc[com_i] 
        com_num = UST_comune.loc[UST_comune["Numero del Distretto"] == dis_num, "Numero UST del Comune"].iloc[com_i]  
        filename_gdf = "raw-gdf-"+str(com_num)+".csv"
        filename_data = "DATA_"+str(com_num)+".csv"

        if pathlib.Path(CANTON_CSV_DIRECTORY/filename_gdf).exists():
            data_nonres = pd.read_csv(CANTON_CSV_DIRECTORY/filename_gdf , header=0, index_col=0)
            #data_rea = data_rea_dirty.drop_duplicates(keep="first")
        elif pathlib.Path(CANTON_CSV_DIRECTORY/filename_data).exists():
            data_nonres = pd.read_csv(CANTON_CSV_DIRECTORY/filename_data , header=0, index_col=0)
            #data_rea = data_rea_dirty.drop_duplicates(keep="first")

        # Setting up the base year to 2020, so drop buildings that where buit after 2020 
        data_nonres = data_nonres.drop(data_nonres[(data_nonres["gbauj"]==2021)|(data_nonres["gbauj"]==2022)|(data_nonres["gbauj"]==2022)].index)

        # Keep only existing residential buildings
        data_nonres = data_nonres.loc[data_nonres["gstat"] == 1004]
        data_nonres = data_nonres[REA_CODES_DESIRED_LC]
        data_nonres = data_nonres.loc[(data_nonres["gkat"] == 1040) | (data_nonres["gkat"] == 1060) | (data_nonres["gkat"] == 1080)]

        data_nonres["garea_fill"] = data_nonres["garea"]
        data_nonres["gbaup_fill"] = data_nonres["gbaup"]
        data_nonres["gastw_fill"] = data_nonres["gastw"]
        data_nonres["gklas_fill"] =  data_nonres["gklas"]

        data_nonres = data_nonres.reset_index()

        age_group = data_nonres[["gkat", "gklas", "gbaup"]]
        age_pivot = pd.pivot_table(age_group, values='gbaup', index=['gkat'], columns=['gklas'], aggfunc=np.median)
        age_pivot = age_pivot.round(0)
        age_col = list(age_pivot.columns)
        for code in GKLAS:
            if code in age_col:
                pass
            else:                
                age_pivot[code] = 8016.0

        size_group = data_nonres[["gkat", "gklas", "garea"]]
        size_pivot = pd.pivot_table(size_group, values='garea', index=['gkat'], columns=['gklas'], aggfunc=np.median)
        size_pivot = size_pivot.round(0)
        size_col = list(size_pivot.columns)

        for g in range(len(data_nonres)):
            if np.isnan(data_nonres.at[g,"gklas"]): 
                if (data_nonres.at[g,"gkat"] == 1040.0):
                    data_nonres.loc[g,"gklas_fill"] = 1211.0 #
                elif (data_nonres.at[g,"gkat"] == 1060.0):
                    data_nonres.loc[g,"gklas_fill"] =1271.0 #
                elif (data_nonres.at[g,"gkat"] == 1080.0):
                    data_nonres.loc[g,"gklas_fill"] =1273.0 # 
            else:
                pass

            if np.isnan(data_nonres.at[g,"gbaup"]):             
                data_nonres.at[g,"gbaup_fill"] = age_pivot.loc[data_nonres.at[g,"gkat"],data_nonres.at[g,"gklas_fill"]]

            if np.isnan(data_nonres.at[g,"garea"]):
                if data_nonres.at[g,"gklas_fill"] in GKLAS_NOHTG:
                    data_nonres.at[g,"garea_fill"] = 0
                else:    
                    data_nonres.at[g,"garea_fill"] = size_pivot.loc[(data_nonres.at[g,"gkat"],data_nonres.at[g,"gklas_fill"])]


        data_nonres["garea_fill"] = data_nonres["garea_fill"].astype(np.int64)
        data_nonres["gklas_fill"] = data_nonres["gklas_fill"].astype(np.int64)
        data_nonres["gbaup_fill"] = data_nonres["gbaup_fill"].astype(np.int64)
        data_nonres["gastw_fill"] = data_nonres["gastw_fill"].fillna(value = 1)


        #drop duplicate egids
        data_nonres = data_nonres.drop_duplicates(subset=['egid'], ignore_index = True)


        for g in range(len(data_nonres)):
            data_nonres.at[g,'k_SH'] = data_sh.loc[(data_nonres.at[g,"gbaup_fill"],str(data_nonres.at[g,"gklas_fill"]))]
            data_nonres.at[g,'k_DHW'] = data_dhw.loc[(data_nonres.at[g,"gbaup_fill"],str(data_nonres.at[g,"gklas_fill"]))]
            data_nonres.at[g,'k_fehh'] = fehh.loc[(data_nonres.at[g,"gbaup_fill"],str(data_nonres.at[g,"gklas_fill"]))]

        district_nonres = pd.concat([district_nonres,data_nonres],ignore_index=True)

    # calculating building attributes and demand
    district_nonres['ERA'] = district_nonres['garea_fill']*district_nonres['gastw_fill']*NET_ERA_SHARE

    district_nonres['SH'] = district_nonres['k_SH']*district_nonres['ERA']*HDD.at["r_HDD",DISTRICT]        
    district_nonres['DHW'] = district_nonres['k_DHW']*district_nonres['ERA']

    agr_data = district_nonres.loc[district_nonres["gklas_fill"] == (AGR_code)]
    agr_htg = agr_data["SH"].sum()
    agr_dhw = agr_data["DHW"].sum() 

    ind_data = district_nonres.loc[district_nonres["gklas_fill"] == (IND_code)]
    ind_htg = ind_data["SH"].sum()
    ind_dhw = ind_data["DHW"].sum() 

    ser_data = district_nonres.loc[district_nonres["gklas_fill"].isin(SER_code)]
    ser_htg = ser_data["SH"].sum()
    ser_dhw = ser_data["DHW"].sum()

    d_nonres = pd.DataFrame([agr_htg, agr_dhw, ind_htg, ind_dhw, ser_htg, ser_dhw], index=["agr_htg", "agr_dhw", "ind_htg", "ind_dhw", "ser_htg", "ser_dhw"], columns=[DISTRICT])
    TI_nonres = pd.concat([TI_nonres,d_nonres], axis=1)
    
    district_nonres = district_nonres[0:0]
    
TI_nonres = pd.concat([TI_nonres,S123])
TI_nonres["TI"] = TI_nonres.sum(axis=1)

path = str(DATA_DIRECTORY)+ "/results/non_residential_htg_demand.csv"
filepath = pathlib.Path(path)  
filepath.parent.mkdir(parents=True, exist_ok=True)  
TI_nonres.to_csv(filepath, sep=";", encoding='utf-8-sig', index_label='index')
print(TI_nonres)
TI_nonres = TI_nonres[0:0]

Bellinzona
Blenio
Leventina
Locarno
Lugano
Mendrisio
Riviera
Vallemaggia
               Bellinzona        Blenio     Leventina       Locarno   
agr_htg      0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  \
agr_dhw      0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
ind_htg      5.431119e+07  1.237361e+06  3.609950e+07  2.780999e+07   
ind_dhw      4.626261e+05  8.125200e+03  2.366739e+05  3.720699e+05   
ser_htg      9.808703e+07  1.518085e+07  2.545904e+07  9.281103e+07   
ser_dhw      1.079349e+07  2.038581e+06  3.093638e+06  2.045746e+07   
S1_agr_elec  1.072536e+07  7.643360e+06  7.088600e+06  1.208144e+07   
S2_ind_elec  9.449475e+07  1.628690e+07  2.304750e+07  1.550328e+08   
S3_ser_elec  8.608058e+07  7.119780e+06  1.166668e+07  1.187074e+08   

                   Lugano     Mendrisio       Riviera   Vallemaggia   
agr_htg      0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  \
agr_dhw      0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
ind

In [23]:
# 4 - Calculate the heating demand for agriculture, industry, and services - WORKS!

## CREATE EMPTY DATAFRAMES
TI_nonres = pd.DataFrame()
district_nonres = pd.DataFrame()

for dis_num in UST_district:  
    DISTRICT = str(UST_comune.loc[(UST_comune["Numero del Distretto"] == dis_num), "Nome del Distretto"].iloc[0])
    print(DISTRICT)
 
    ic = len(UST_comune.loc[(UST_comune["Numero del Distretto"] == dis_num)])

    for com_i in range(ic):
        com_name = UST_comune.loc[UST_comune["Numero del Distretto"] == dis_num, "Nome del Comune"].iloc[com_i] 
        com_num = UST_comune.loc[UST_comune["Numero del Distretto"] == dis_num, "Numero UST del Comune"].iloc[com_i]  
        filename_gdf = "raw-gdf-"+str(com_num)+".csv"
        filename_data = "DATA_"+str(com_num)+".csv"

        if pathlib.Path(CANTON_CSV_DIRECTORY/filename_gdf).exists():
            data_nonres = pd.read_csv(CANTON_CSV_DIRECTORY/filename_gdf , header=0, index_col=0)
            #data_rea = data_rea_dirty.drop_duplicates(keep="first")
        elif pathlib.Path(CANTON_CSV_DIRECTORY/filename_data).exists():
            data_nonres = pd.read_csv(CANTON_CSV_DIRECTORY/filename_data , header=0, index_col=0)
            #data_rea = data_rea_dirty.drop_duplicates(keep="first")

        # Setting up the base year to 2020, so drop buildings that where buit after 2020 
        data_nonres = data_nonres.drop(data_nonres[(data_nonres["gbauj"]==2021)|(data_nonres["gbauj"]==2022)|(data_nonres["gbauj"]==2022)].index)

        # Keep only existing residential buildings
        data_nonres = data_nonres.loc[data_nonres["gstat"] == 1004]
        data_nonres = data_nonres[REA_CODES_DESIRED_LC]
        data_nonres = data_nonres.loc[(data_nonres["gkat"] == 1040) | (data_nonres["gkat"] == 1060) | (data_nonres["gkat"] == 1080)]

        data_nonres["garea_fill"] = data_nonres["garea"]
        data_nonres["gbaup_fill"] = data_nonres["gbaup"]
        data_nonres["gastw_fill"] = data_nonres["gastw"]
        data_nonres["gklas_fill"] =  data_nonres["gklas"]

        data_nonres = data_nonres.reset_index()

        age_group = data_nonres[["gkat", "gklas", "gbaup"]]
        age_pivot = pd.pivot_table(age_group, values='gbaup', index=['gkat'], columns=['gklas'], aggfunc=np.median)
        age_pivot = age_pivot.round(0)
        age_col = list(age_pivot.columns)
        for code in GKLAS:
            if code in age_col:
                pass
            else:                
                age_pivot[code] = 8016.0

        size_group = data_nonres[["gkat", "gklas", "garea"]]
        size_pivot = pd.pivot_table(size_group, values='garea', index=['gkat'], columns=['gklas'], aggfunc=np.median)
        size_pivot = size_pivot.round(0)
        size_col = list(size_pivot.columns)

        for g in range(len(data_nonres)):
            if np.isnan(data_nonres.at[g,"gklas"]): 
                if (data_nonres.at[g,"gkat"] == 1040.0):
                    data_nonres.loc[g,"gklas_fill"] = 1211.0 #
                elif (data_nonres.at[g,"gkat"] == 1060.0):
                    data_nonres.loc[g,"gklas_fill"] =1271.0 #
                elif (data_nonres.at[g,"gkat"] == 1080.0):
                    data_nonres.loc[g,"gklas_fill"] =1273.0 # 
            else:
                pass

            if np.isnan(data_nonres.at[g,"gbaup"]):             
                data_nonres.at[g,"gbaup_fill"] = age_pivot.loc[data_nonres.at[g,"gkat"],data_nonres.at[g,"gklas_fill"]]

            if np.isnan(data_nonres.at[g,"garea"]):
                if data_nonres.at[g,"gklas_fill"] in GKLAS_NOHTG:
                    data_nonres.at[g,"garea_fill"] = 0
                else:    
                    data_nonres.at[g,"garea_fill"] = size_pivot.loc[(data_nonres.at[g,"gkat"],data_nonres.at[g,"gklas_fill"])]


        data_nonres["garea_fill"] = data_nonres["garea_fill"].astype(np.int64)
        data_nonres["gklas_fill"] = data_nonres["gklas_fill"].astype(np.int64)
        data_nonres["gbaup_fill"] = data_nonres["gbaup_fill"].astype(np.int64)
        data_nonres["gastw_fill"] = data_nonres["gastw_fill"].fillna(value = 1)


        #drop duplicate egids
        data_nonres = data_nonres.drop_duplicates(subset=['egid'], ignore_index = True)


        for g in range(len(data_nonres)):
            data_nonres.at[g,'k_SH'] = data_sh.loc[(data_nonres.at[g,"gbaup_fill"],str(data_nonres.at[g,"gklas_fill"]))]
            data_nonres.at[g,'k_DHW'] = data_dhw.loc[(data_nonres.at[g,"gbaup_fill"],str(data_nonres.at[g,"gklas_fill"]))]
            data_nonres.at[g,'k_fehh'] = fehh.loc[(data_nonres.at[g,"gbaup_fill"],str(data_nonres.at[g,"gklas_fill"]))]

        district_nonres = pd.concat([district_nonres,data_nonres],ignore_index=True)

    # calculating building attributes and demand
    district_nonres['ERA'] = district_nonres['garea_fill']*district_nonres['gastw_fill']*NET_ERA_SHARE

    district_nonres['SH'] = district_nonres['k_SH']*district_nonres['ERA']*HDD.at["r_HDD",DISTRICT]        
    district_nonres['DHW'] = district_nonres['k_DHW']*district_nonres['ERA']

    agr_data = district_nonres.loc[district_nonres["gklas_fill"] == (AGR_code)]
    agr_htg = agr_data["SH"].sum()
    agr_dhw = agr_data["DHW"].sum() 

    ind_data = district_nonres.loc[district_nonres["gklas_fill"] == (IND_code)]
    ind_htg = ind_data["SH"].sum()
    ind_dhw = ind_data["DHW"].sum() 

    ser_data = district_nonres.loc[district_nonres["gklas_fill"].isin(SER_code)]
    ser_htg = ser_data["SH"].sum()
    ser_dhw = ser_data["DHW"].sum()

    d_nonres = pd.DataFrame([agr_htg, agr_dhw, ind_htg, ind_dhw, ser_htg, ser_dhw], index=["agr_htg", "agr_dhw", "ind_htg", "ind_dhw", "ser_htg", "ser_dhw"], columns=[DISTRICT])
    TI_nonres = pd.concat([TI_nonres,d_nonres], axis=1)
    
    district_nonres = district_nonres[0:0]
    
TI_nonres["TI"] = TI_nonres.sum(axis=1)

path = str(DATA_DIRECTORY)+ "/results/non_residential_demand.csv"
filepath = pathlib.Path(path)  
filepath.parent.mkdir(parents=True, exist_ok=True)  
TI_nonres.to_csv(filepath, sep=";", encoding='utf-8-sig', index_label='index')
print(TI_nonres)
TI_nonres = TI_nonres[0:0]

Bellinzona
Blenio
Leventina
Locarno
Lugano
Mendrisio
Riviera
Vallemaggia
           Bellinzona        Blenio     Leventina       Locarno        Lugano   
agr_htg  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  \
agr_dhw  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
ind_htg  5.431119e+07  1.237361e+06  3.609950e+07  2.780999e+07  6.019938e+07   
ind_dhw  4.626261e+05  8.125200e+03  2.366739e+05  3.720699e+05  7.283565e+05   
ser_htg  9.808703e+07  1.518085e+07  2.545904e+07  9.281103e+07  2.724004e+08   
ser_dhw  1.079349e+07  2.038581e+06  3.093638e+06  2.045746e+07  4.222838e+07   

            Mendrisio       Riviera   Vallemaggia            TI  
agr_htg  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  
agr_dhw  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  
ind_htg  4.297798e+07  1.322245e+07  2.500217e+06  2.383581e+08  
ind_dhw  5.835366e+05  8.593830e+04  2.272950e+04  2.500056e+06  
ser_htg  6.877692e+07  1.4720

### RESIDENTIAL

In [15]:
#  5 - RESIDENTIAL ELECTRICITY AND HEAITNG DEMAND PER DISTRICT - Getting the number of households according to ERA (EWID) - WORKS!
## NEED TO DOUBLE CHECK the gklass_fill argument

## CREATE EMPTY DATAFRAMES
TI_res = pd.DataFrame()
district_res = pd.DataFrame()
ewid_count = pd.DataFrame()


for dis_num in UST_district:  
    DISTRICT = str(UST_comune.loc[(UST_comune["Numero del Distretto"] == dis_num), "Nome del Distretto"].iloc[0])
    print(DISTRICT)
#    com_name = UST_comune.loc[UST_comune["Numero del Distretto"] == dis_num, "Nome del Comune"] 
#    com_num = UST_comune.loc[UST_comune["Numero del Distretto"] == dis_num, "Numero UST del Comune"] #
    num = round(avg_hh_MFH.at["MFH_avg_households",DISTRICT])

    MFH_hh_list = generate_number_list(num)
    MFH_avg = str(MFH_hh_list)
    
    ic = len(UST_comune.loc[(UST_comune["Numero del Distretto"] == dis_num)])

    for com_i in range(ic):
        com_name = UST_comune.loc[UST_comune["Numero del Distretto"] == dis_num, "Nome del Comune"].iloc[com_i] 
        com_num = UST_comune.loc[UST_comune["Numero del Distretto"] == dis_num, "Numero UST del Comune"].iloc[com_i]  
        filename_gdf = "raw-gdf-"+str(com_num)+".csv"
        filename_data = "DATA_"+str(com_num)+".csv"

        if pathlib.Path(CANTON_CSV_DIRECTORY/filename_gdf).exists():
            data_res = pd.read_csv(CANTON_CSV_DIRECTORY/filename_gdf , header=0, index_col=0)
            #data_res = data_res_dirty.drop_duplicates(keep="first")
        elif pathlib.Path(CANTON_CSV_DIRECTORY/filename_data).exists():
            data_res = pd.read_csv(CANTON_CSV_DIRECTORY/filename_data , header=0, index_col=0)
            #data_res = data_res_dirty.drop_duplicates(keep="first")

        # Keep only existing residential buildings
        data_res = data_res.loc[data_res["gstat"] == 1004]
        data_res = data_res.loc[(data_res["gkat"] == 1020) | (data_res["gkat"] == 1030)]
        data_res = data_res[REA_CODES_DESIRED_LC]
        
        # Setting up the base year to 2020, so drop buildings that where buit after 2020 
        data_res = data_res.drop(data_res[(data_res["gbauj"]==2021)|(data_res["gbauj"]==2022)|(data_res["gbauj"]==2022)].index)

        data_res["garea_fill"] = data_res["garea"]
        data_res["gbaup_fill"] = data_res["gbaup"]
        data_res["gastw_fill"] = data_res["gastw"]
        data_res["gklas_fill"] =  data_res["gklas"]
        
        data_res = data_res.reset_index()

        age_group = data_res[["gkat", "gklas", "gbaup"]]
        age_pivot = pd.pivot_table(age_group, values='gbaup', index=['gkat'], columns=['gklas'], aggfunc=np.median)
        age_pivot = age_pivot.round(0)
        age_col = list(age_pivot.columns)
        for code in GKLAS:
            if code in age_col:
                pass
            else:                
                age_pivot[code] = 8016.0

        size_group = data_res[["gkat", "gklas", "garea"]]
        size_pivot = pd.pivot_table(size_group, values='garea', index=['gkat'], columns=['gklas'], aggfunc=np.median)
        size_pivot = size_pivot.round(0)
        size_col = list(size_pivot.columns)
        
        for g in range(len(data_res)):
            if np.isnan(data_res.at[g,"gklas"]): 
            ## CHECK IF THIS IS A MISTAKE AND IT IS CHANGING ALL VALUES NSTEAD OF MISSING ONES 
                if (data_res.at[g,"gkat"] == 1020):
                    data_res.loc[g,"gklas_fill"] = 1110
                elif (data_res.at[g,"gkat"] == 1030):
                    data_res.loc[g,"gklas_fill"] = 1122 # 63% of known 1040

            data_res["ewid"] = data_res["ewid"].fillna(value = "['no']")

                    
            if (data_res.at[g,"ewid"]=="['no']"):
                if (data_res.at[g,"gklas_fill"] == 1110):
                    data_res.at[g,"ewid"] = "['1']"
                elif (data_res.at[g,"gklas_fill"] == 1121):
                    data_res.at[g,"ewid"] = "['1', '2']"
                elif (data_res.at[g,"gklas_fill"] == 1122):
                    data_res.at[g,"ewid"] = MFH_avg # GET real median
            else:
                pass
      
            if np.isnan(data_res.at[g,"gbaup"]):             
                data_res.at[g,"gbaup_fill"] = age_pivot.loc[data_res.at[g,"gkat"],data_res.at[g,"gklas_fill"]]

            if np.isnan(data_res.at[g,"garea"]):
                if data_res.at[g,"gklas_fill"] in GKLAS_NOHTG:
                    data_res.at[g,"garea_fill"] = 0
                else:    
                    if data_res.at[g,"gklas_fill"] in size_col:
                        pass
                    else:   
                        if data_res.at[g,"gklas_fill"] == 1130.0:
                            size_pivot.loc[1040.0,1130.0] = size_pivot.loc[1040.0, 1211.0]
                        else:
                            pass
                    data_res.at[g,"garea_fill"] = size_pivot.loc[(data_res.at[g,"gkat"],data_res.at[g,"gklas_fill"])]


        data_res["garea_fill"] = data_res["garea_fill"].astype(np.int64)
        data_res["gklas_fill"] = data_res["gklas_fill"].astype(np.int64)
        data_res["gbaup_fill"] = data_res["gbaup_fill"].astype(np.int64)
        data_res["gastw_fill"] = data_res["gastw_fill"].fillna(value = 1)
        
        for g in range(len(data_res)):
            ewid_count.at[g,"ewid"] = ast.literal_eval(data_res.at[g,"ewid"])      

        data_res["ewid_len"] = ewid_count.ewid.apply(lambda x: len(x))
        
        # sum the ewid_len per egid
        grouped = data_res.groupby('egid')["ewid_len"].sum().reset_index()
        grouped = pd.DataFrame(grouped)
        
        #drop duplicate egids
        data_res = data_res.drop_duplicates(subset=['egid'])
        
        # Merge dataframes based on column 'egid'
        data_res = data_res.merge(grouped, on='egid', how='left', suffixes=('_original', '_replacement'))

        # Update 'ewid' values based on the replacement values
        data_res['ewid_len'] = data_res['ewid_len_replacement'].fillna(data_res['ewid_len_original'])

        # Drop the original and replacement columns
        data_res = data_res.drop(columns=['ewid_len_original', 'ewid_len_replacement'])
                
        for g in range(len(data_res)):
            data_res.at[g,'k_SH'] = data_sh.loc[(data_res.at[g,"gbaup_fill"],str(data_res.at[g,"gklas_fill"]))]
            data_res.at[g,'k_DHW'] = data_dhw.loc[(data_res.at[g,"gbaup_fill"],str(data_res.at[g,"gklas_fill"]))]
            data_res.at[g,'k_fehh'] = fehh.loc[(data_res.at[g,"gbaup_fill"],str(data_res.at[g,"gklas_fill"]))]

        district_res = pd.concat([district_res,data_res],ignore_index=True)
    
    sample_size = int(len(district_res)*(1-UNOCC))

    # Take a random sample of 90% of the rows
    district_res_occ = district_res.sample(n=sample_size, random_state=1) #random state can be changed, otherwise always same sample
    district_res_occ.reset_index(drop=True, inplace = True)
    
    d_hh = (district_res_occ["ewid_len"].sum())
    
    #d_hh = d_hh*hh_stats.at["hh2023_diff",DISTRICT] #"hh2023_diff" = ratio of residences in 2020/2023
    d_hh_p = d_hh*(1-hh_stats.at["hh2020_p_ratio",DISTRICT]) # (1-"hh2020_p_ratio"= primary residences 
    d_hh_s = d_hh*(hh_stats.at["hh2020_p_ratio",DISTRICT])  # "hh2020_p_ratio" = ratio of secondary residences 
    
    num_rows = len(district_res_occ)
    num_ones = round(len(district_res_occ)*(1-hh_stats.at["hh2020_p_ratio",DISTRICT]))

    rand_values = np.array([1] * num_ones + [1/4] * (num_rows - num_ones))
    np.random.shuffle(rand_values)

    # Assign the generated random values to the "primary" column
    district_res_occ['primary'] = rand_values

    # calculating building attributes and demand
    district_res_occ['ERA'] = district_res_occ['garea_fill']*district_res_occ['gastw_fill']*NET_ERA_SHARE
    
    for g in range(len(district_res_occ)):
        if district_res_occ.at[g,'primary'] == 1:
            district_res_occ.at[g,'SH'] = district_res_occ.at[g,'k_SH']*district_res_occ.at[g,'ERA']*district_res_occ.at[g,'primary']*HDD.at["r_HDD",DISTRICT]
        else:
            district_res_occ.at[g,'SH'] = district_res_occ.at[g,'k_SH']*district_res_occ.at[g,'ERA']*(district_res_occ.at[g,'primary']/3)*HDD.at["r_HDD",DISTRICT]
            
    district_res_occ['DHW'] = district_res_occ['k_DHW']*district_res_occ['ERA']*district_res_occ['primary']
    district_res_occ['ELEC'] = district_res_occ['ewid_len']*AVG_HH_ELEC_TI*district_res_occ['primary']

    
    d_elec = district_res_occ["ELEC"].sum() 
    d_sh = district_res_occ["SH"].sum() 
    d_dhw = district_res_occ["DHW"].sum() 
    
    d = pd.DataFrame([d_hh, d_hh_p, d_hh_s, d_elec, d_sh, d_dhw], index=["hh","hh_p","hh_s","res_elec","res_sh","res_dhw"], columns=[DISTRICT])

    TI_res = pd.concat([TI_res,d], axis=1)
    district_res = district_res[0:0]

TI_res["TI"] = TI_res.sum(axis=1)

path = str(DATA_DIRECTORY)+ "/results/residential_demand.csv"
filepath = pathlib.Path(path)  
filepath.parent.mkdir(parents=True, exist_ok=True)  
TI_res.to_csv(filepath, sep=";", encoding='utf-8-sig', index_label='index')
print(TI_res)
TI_res = TI_res[0:0]

Bellinzona
Blenio
Leventina
Locarno
Lugano
Mendrisio
Riviera
Vallemaggia
            Bellinzona        Blenio     Leventina       Locarno   
hh        3.227000e+04  6.926000e+03  9.348000e+03  5.669300e+04  \
hh_p      2.710680e+04  2.576472e+03  4.440300e+03  3.248509e+04   
hh_s      5.163200e+03  4.349528e+03  4.907700e+03  2.420791e+04   
res_elec  9.945820e+07  1.278447e+07  1.991543e+07  1.344106e+08   
res_sh    3.881450e+08  5.755207e+07  9.700613e+07  3.163994e+08   
res_dhw   6.982699e+07  7.166092e+06  1.262213e+07  9.322201e+07   

                Lugano     Mendrisio       Riviera   Vallemaggia            TI  
hh        9.500600e+04  2.990200e+04  6.178000e+03  6.648000e+03  2.429710e+05  
hh_p      7.714487e+04  2.466915e+04  5.047426e+03  2.645904e+03  1.761160e+05  
hh_s      1.786113e+04  5.232850e+03  1.130574e+03  4.002096e+03  6.685499e+04  
res_elec  2.861946e+08  9.104398e+07  1.853333e+07  1.278272e+07  6.751233e+08  
res_sh    8.142494e+08  2.787720e+08  8.98640

# TESTS